# highRES

What is needed?

solar + wind

	vre_areas_2013_dev
		areas
			technology.region.grid value `capacity_matrix_solar`
				not for hydrores?
                
	vre_2013.gdx
		one value for each hour, technology, country, gridcell

Capacity factors:
* capacity factors per hour, technology, country `capacity_factor = cutout.pv(panel=panel, orientation=orientation, matrix=capacity_matrix_solar, capacity_factor=True, per_unit=True, index=europe.index)`
* capacity factor per hour, technology, gridcell `maybe_progressbar(da, show_progress)`
    * FIXME important to keep the indexing proper.

There are two areas I can think of:
* Area of selection
    * Area of the shape (country) `croatia.geometry.item().area`
    * Area of the grid cell `area = xr.DataArray(area, dims=('spatial'))`
* Available area to build on in selection
    * Area that is available to be built on in the country `capacity_matrix.sum(spatial)`
    * Area that is available to be built on in the gridcell `capacity_matrix.sum(index)`
    * Area that is available to be built on in the gridcell in the country `capacity_matrix`

For the proposed comparison, areas and capacity factors are needed on two levels.
1. Shape level (countries)
2. Gridcell level (model does the picking)
    here we use the debug mode of spyder

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import io
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import xarray as xr
from atlite.gis import ExclusionContainer, shape_availability
from shapely.geometry import Polygon

In [ ]:
atlite.__version__

In [ ]:
xr.__version__

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

# Desired regions

In [ ]:
highRES_new_countries = [
    "NO020",
    "NO060",
    "NO071",
    "NO074",
    "NO081",
    "NO082",
    "NO091",
    "NO092",
    "NO0A1",
    "NO0A2",
    "NO0A3",
]

Here we drop Norway (because we have the Fylke) and Iceland (because we do not have it).

In [ ]:
desired_regions = highRES_new_countries
desired_regions.sort()
desired_regions

In [ ]:
ssb_to_nuts = {
    "NO03": "NO081",
    "NO30": "NO082",
    "NO34": "NO020",
    "NO38": "NO091",
    "NO42": "NO092",
    "NO11": "NO0A1",
    "NO46": "NO0A2",
    "NO15": "NO0A3",
    "NO50": "NO060",
    "NO18": "NO071",
    "NO54": "NO074",
}
ssb_to_nuts

In [ ]:
nuts_to_ssb = {value: key for key, value in ssb_to_nuts.items()}
nuts_to_ssb

## Shape level
To get the desired data at shape level, the first input is the shapefile itself.
capacity_factor_raw = da ohne mean in convert and aggregate

In [ ]:
#NUTS_RG_60M_2021_4326_NO_LEVL_3_REST_LEVL_0.geojson
#url = "https://uio-my.sharepoint.com/:u:/g/personal/maximilr_uio_no/EYYGvCuWhlFFvjLe0JKlz1ABwq1eBHRa72zbivOJtK6pQQ?e=LlFIDB&download=1"
#urlData = requests.get(url).content
#spurl = io.StringIO(urlData.decode("utf-8"))
#europe = (
#    gpd.read_file(spurl)
#    .replace({"UK": "GB", "EL": "GR"})
#    .rename(columns={"id": "index"})
#    .loc[:, ["index", "geometry"]]
#    .sort_values("index")
#    .set_index("index")
#    .drop(["AL", "ME", "MK", "NO0B1", "NO0B2", "RS", "TR", "LI"])
#)
#europe = gpd.read_file("NUTS_RG_10M_2021_4326_europe_LEVL_0.geojson").set_index("index")

In [ ]:
sharedinputpath = snakemake.params.sharedinputpath

In [ ]:
file_name = (
    sharedinputpath
    / "geodata/onshore/NUTS_RG_60M_2021_4326_NO_LEVL_3_REST_LEVL_0.geojson"
)
file_name

In [ ]:
file_name = (
    sharedinputpath
    / "geodata/onshore/shapes/NUTS_RG_60M_2021_4326_NO_LEVL_3_REST_LEVL_0.geojson"
)
europe = (
    gpd.read_file(file_name)
    .replace({"EL": "GR"})
    .rename(columns={"id": "index"})
    .loc[:, ["index", "geometry"]]
    .sort_values("index")
    .set_index("index")
    .drop(["AL", "IS", "CY", "MT", "ME", "MK", "NO0B1", "NO0B2", "RS", "TR", "LI"])
    .loc[desired_regions]
)

In [ ]:
europe

In [ ]:
year = snakemake.wildcards.year

In [ ]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)

In [ ]:
# europe = europe.drop(index = 'IS') #removes Iceland as it is not included in the ERA5 cutout
# europe = europe.loc[["AT","DE"]]
europe.plot(figsize=(15, 15));

In [ ]:
cutout = atlite.Cutout(path=sharedinputpath / ("weatherdata/europe_" + str(year) + ".nc"))

In [ ]:
cutout.prepare()

In [ ]:
europe = europe.sort_index()

# Hydro

In [ ]:
fn = snakemake.input.eiahydrogen

In [ ]:
# in billion KWh/a = TWh/a
eia_hydro_gen = (
    pd.read_csv(fn, skiprows=4, index_col=1, na_values=[" ", "--"])
    .drop(["Unnamed: 0", "Unnamed: 2"], axis=1)
    .dropna(how="all")
)
eia_hydro_gen

In [ ]:
from countrycode.countrycode import countrycode

In [ ]:
countries_iso2c = countrycode(
    eia_hydro_gen.index.values, origin="country_name", target="iso2c"
)
countries_iso2c

In [ ]:
eia_hydro_gen.index = pd.Index(countries_iso2c, name="countries")
eia_hydro_gen

In [ ]:
eia_hydro_gen.rename(index={"Kosovo": "KV"}, inplace=True)
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen.T
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen * 1e6  # in MWh/a
eia_hydro_gen

In [ ]:
europe.index

In [ ]:
eia_stats = eia_hydro_gen.reindex(columns=europe.index)
eia_stats

In [ ]:
fraction = (eia_hydro_gen.NO)[str(year)]
fraction

In [ ]:
eia_stats.columns

### Calculating Norwegian fractions

In [ ]:
hydro_gen = pd.read_csv(
    snakemake.input.vannkraft,
    encoding="latin",
    skiprows=2,
    sep=";",
    decimal=",",
    index_col=False,
)

In [ ]:
hydro_gen

In [ ]:
hydro_fractions = (
    hydro_gen.rename(
        columns={
            "Midl. årsproduksjon (ref. 1981-2010) [GWh]": "average_generation_GWh",
            "Maks ytelse [MW]": "installed_capacity_MW",
        }
    )
    .groupby("Fylke")
    .sum()
    .assign(
        share_gen=lambda x: x.average_generation_GWh / x.average_generation_GWh.sum(),
        share_cap=lambda x: x.installed_capacity_MW / x.installed_capacity_MW.sum(),
        NUTS3_ID=[
            "NO092",
            "NO020",
            "NO0A3",
            "NO071",
            "NO081",
            "NO0A1",
            "NO074",
            "NO060",
            "NO091",
            "NO0A2",
            "NO082",
        ],
    )
    .set_index("NUTS3_ID")
    .sort_index()
)
hydro_fractions

In [ ]:
for zone in highRES_new_countries:
    print(zone)
    eia_stats.loc[str(year), zone] = fraction * hydro_fractions.loc[zone, "share_gen"]

In [ ]:
eia_stats = eia_stats.loc[[year], :]

In [ ]:
europe2 = europe.copy()

In [ ]:
europe2.index.name = "countries"

In [ ]:
europe2

In [ ]:
inflow = cutout.runoff(
    shapes=europe2,
    smooth=True,
    lower_threshold_quantile=True,
    normalize_using_yearly=eia_stats
    # , per_unit = True
)

This is our data for hydro_inflow_2013.gdx

In [ ]:
# scale the inflow by installed capacities
df_installed_cap = pd.read_csv(
    snakemake.input.hydroinstalledcap,
    sep="\t",
    usecols=["Year", "AreaName", "MapCode", "ProductionType", "highRES_Europe"],
)
df_installed_cap = df_installed_cap.loc[
    df_installed_cap["ProductionType"] != "Hydro Pumped Storage"
]

In [ ]:
df_installed_cap.MapCode.unique()

In [ ]:
df_cap_shares = (
    df_installed_cap.loc[:, ["MapCode", "ProductionType", "highRES_Europe"]]
    .set_index(["ProductionType", "MapCode"])
    .unstack("ProductionType")
    .loc[:, "highRES_Europe"]
    .assign(
        Hydro=lambda x: x["Hydro Run-of-river and poundage"]
        + x["Hydro Water Reservoir"],
        share_ror=lambda x: x["Hydro Run-of-river and poundage"] / x["Hydro"],
        share_res=lambda x: x["Hydro Water Reservoir"] / x["Hydro"],
    )
    .rename(index=nuts_to_ssb)
    .loc[:, ["share_ror", "share_res"]]
)
df_cap_shares

In [ ]:
inflow

In [ ]:
inflow.assign_coords(time=range(0, len(inflow.time))).assign_coords(
    technology="HydroRes"
)

In [ ]:
inflow_res = (
    inflow.assign_coords(time=range(0, len(inflow.time)))
    .assign_coords(technology="HydroRes")
    .expand_dims("technology")
    .stack(final=["time", "countries", "technology"])
    .round(2)
    .to_pandas()
    .reset_index()
)

inflow_res.columns = inflow_res.columns.map(str)
inflow_res = inflow_res.rename(columns={"0": "value"})

In [ ]:
df_cap_shares.loc[:, ["share_res"]]

In [ ]:
inflow_res

In [ ]:
df_cap_shares.loc[inflow_res.replace({"countries": nuts_to_ssb}).countries.unique(), :]

In [ ]:
(
    inflow_res.replace({"countries": nuts_to_ssb})
    .set_index(["countries", "time", "technology"])
    .mul(
        df_cap_shares.rename(columns={"share_res": "value"}),
        axis="index",
        level="countries",
    )
    .drop(columns={"share_ror"})
    .reset_index()
    .set_index("time")
    .round(0)
    .to_csv(snakemake.output["hydroresinfl"])
)

Next, create the GDX file:
`/opt/gams/gams36.2_linux_x64_64_sfx/csv2gdx hydro_inflow_2013.csv output=hydro_inflow_2013.gdx ID=hydro_inflow Index='(1,2,3)' Value='(4)' UseHeader=True StoreZero=True`

## ROR capfac

New work by Oskar

In [ ]:
df_installed_cap.MapCode.unique()

In [ ]:
df_installed_ror = (
    df_installed_cap.loc[
        df_installed_cap["ProductionType"] == "Hydro Run-of-river and poundage"
    ]
    .set_index("MapCode")
    .drop(columns={"Year", "AreaName", "ProductionType"})
    .loc[:, "highRES_Europe"]
)

In [ ]:
df_installed_ror = df_installed_ror[df_installed_ror > 0]

In [ ]:
df_installed_ror

In [ ]:
ror_capfac = (
    inflow.to_pandas()
    .rename(columns=nuts_to_ssb)
    .T
    .stack()
    .to_frame()
    .rename(columns={0:'value'})
    .mul((
        df_cap_shares
        .reset_index()
        .rename(columns={'MapCode' : 'countries', 'share_ror' : 'value'})
        .set_index('countries')
        .loc[:,['value']])
        )
    .div(df_installed_ror, axis="index", level="countries")
   .loc[:,'value']
)

In [ ]:
ror_capfac = ror_capfac.dropna()

In [ ]:
ror_capfac = ror_capfac.where(ror_capfac <= 1, 1)

In [ ]:
ror_capfac.unstack().T.plot(subplots=True,figsize=(10,50),ylim=(0,1.1))

In [ ]:
capacity_factor_hydro_ror = ror_capfac.to_xarray().rename({"countries": "index"})

In [ ]:
capacity_factor_hydro_ror

In [ ]:
capacity_factor_hydro_ror.name = "HydroRoR"

In [ ]:
capacity_factor_hydro_ror.time.size

In [ ]:
capacity_factor_hydro_ror.assign_coords(
    technology=lambda x: x.name, time=range(0, capacity_factor_hydro_ror.time.size)
).expand_dims(
    "technology"
)  # .to_pandas()

In [ ]:
capacity_factor_hydro_ror.assign_coords(
    technology=lambda x: x.name, time=range(0, capacity_factor_hydro_ror.time.size)
).expand_dims("technology").stack(final=["time", "technology", "index"]).round(
    2
).to_pandas().to_csv(
    snakemake.output["hydrororcapfac"], header=False
)

Next, create the GDX file:
`/opt/gams/gams36.2_linux_x64_64_sfx/csv2gdx vre_1997.csv output=vre_1997.gdx ID=vre_gen Index='(1,2,3)' Value='(4)' UseHeader=True StoreZero=True`